In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split


file_path = 'FS8_PPGUID.csv' 
df = pd.read_csv(file_path, low_memory=False)
df.drop('Unnamed: 24',axis=1, inplace=True)
df['TargetFilename'] = df['TargetFilename'].apply(lambda x: 0 if pd.isnull(x) else len(str(x)))
from sklearn.preprocessing import LabelEncoder
df['LogonId'] = df['LogonId'].fillna(-1)
df['TerminalSessionId'] = df['TerminalSessionId'].fillna(-1)

label_encoder_logon = LabelEncoder()
label_encoder_terminal = LabelEncoder()
df['LogonId'] = df['LogonId'].apply(lambda x: x if x == -1 else label_encoder_logon.fit_transform([x])[0])
df['TerminalSessionId'] = df['TerminalSessionId'].apply(lambda x: x if x == -1 else label_encoder_terminal.fit_transform([x])[0])

df['LogonId'] = df['LogonId'].astype(int)
df['TerminalSessionId'] = df['TerminalSessionId'].astype(int)
df_new = df
df_new = df_new.drop_duplicates()
#df_new.info()
df_new.replace('-', 'null', inplace=True)
#df_new.info()
df_new['CurrentDirectory'] = df_new['CurrentDirectory'].apply(lambda x: -1 if pd.isna(x) or x == 'null' else len(str(x)))
df_new['LogonGuid'] = df_new['LogonGuid'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)


df_new['IntegrityLevel'] = df_new['LogonGuid'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['ParentUser'] = df_new['ParentUser'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['Protocol'] = df_new['Protocol'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['Initiated'] = df_new['Initiated'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['SourceIsIpv6'] = df_new['SourceIsIpv6'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['SourceIp'] = df_new['SourceIp'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['DestinationIsIpv6'] = df_new['DestinationIsIpv6'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['DestinationIp'] = df_new['DestinationIp'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['ParentProcessGuid'] = df_new['ParentProcessGuid'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['SourcePort'] = df_new['SourcePort'].apply(lambda x: -1 if pd.isna(x) or x == 'null' else x)
df_new['DestinationPort'] = df_new['DestinationPort'].apply(lambda x: -1 if pd.isna(x) or x == 'null' else x)
df_new['User'] = df_new['User'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
#df_new.info()
df_new['label'].value_counts()
df_final = df_new
X = df_final.drop(columns=['label'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_benign = X_scaled[df_new['label'] == 0]
X_malicious = X_scaled[df_new['label'] == 1]





X_train_benign, X_test_benign = train_test_split(X_benign, test_size=0.2, random_state=42)

lof = LocalOutlierFactor(n_neighbors=20, contamination='auto', novelty=True)
lof.fit(X_train_benign)

y_test_combined = df_new['label'] 
y_pred_lof = lof.predict(X_scaled)
y_pred_lof = [1 if pred == -1 else 0 for pred in y_pred_lof]

print("Confusion Matrix (Local Outlier Factor):\n", confusion_matrix(y_test_combined, y_pred_lof))
print("Classification Report (Local Outlier Factor):\n", classification_report(y_test_combined, y_pred_lof))

C:\Users\User\AppData\Local\Temp\ipykernel_8820\997760969.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.replace('-', 'null', inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_8820\997760969.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['CurrentDirectory'] = df_new['CurrentDirectory'].apply(lambda x: -1 if pd.isna(x) or x == 'null' else len(str(x)))
C:\Users\User\AppData\Local\Temp\ipykernel_8820\997760969.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

Confusion Matrix (Local Outlier Factor):
 [[271937  12989]
 [ 51323  11790]]
Classification Report (Local Outlier Factor):
               precision    recall  f1-score   support

           0       0.84      0.95      0.89    284926
           1       0.48      0.19      0.27     63113

    accuracy                           0.82    348039
   macro avg       0.66      0.57      0.58    348039
weighted avg       0.77      0.82      0.78    348039



In [3]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report, confusion_matrix

param_grid = {
    'n_neighbors': [5, 10, 20, 30],  
    'novelty': [True],               
    'contamination': np.linspace(0.01, 0.2, 5) 
}

lof_1 = LocalOutlierFactor(novelty=True)

grid_search = GridSearchCV(estimator=lof_1, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)

grid_search.fit(X_train_benign)

best_params = grid_search.best_params_
best_lof = grid_search.best_estimator_

print("Best Parameters found by GridSearchCV:", best_params)

y_pred_lof_best = best_lof.predict(X_scaled)
y_pred_lof_best = [1 if pred == -1 else 0 for pred in y_pred_lof_best]

print("Confusion Matrix (Tuned Local Outlier Factor):\n", confusion_matrix(y_test_combined, y_pred_lof_best))
print("Classification Report (Tuned Local Outlier Factor):\n", classification_report(y_test_combined, y_pred_lof_best))


Fitting 3 folds for each of 20 candidates, totalling 60 fits


C:\Users\User\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
  warnings.warn(


Best Parameters found by GridSearchCV: {'contamination': 0.01, 'n_neighbors': 5, 'novelty': True}
Confusion Matrix (Tuned Local Outlier Factor):
 [[283825   1101]
 [ 62703    410]]
Classification Report (Tuned Local Outlier Factor):
               precision    recall  f1-score   support

           0       0.82      1.00      0.90    284926
           1       0.27      0.01      0.01     63113

    accuracy                           0.82    348039
   macro avg       0.55      0.50      0.46    348039
weighted avg       0.72      0.82      0.74    348039



In [5]:
import numpy as np

X_test_malicious_sampled = X_malicious[:20000]

X_test_combined = np.concatenate([X_test_benign, X_test_malicious_sampled], axis=0)

y_test_combined = np.array([0] * len(X_test_benign) + [1] * len(X_test_malicious_sampled))
print("y_test_combined labels:\n", y_test_combined)

print("First 20 labels of y_test_combined:\n", y_test_combined[:20])

unique, counts = np.unique(y_test_combined, return_counts=True)
print("Label counts using np.unique:")
print(dict(zip(unique, counts)))

from collections import Counter
label_counts = Counter(y_test_combined)
print("Label counts using Counter:")
print(label_counts)

y_test_combined labels:
 [0 0 0 ... 1 1 1]
First 20 labels of y_test_combined:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label counts using np.unique:
{0: 56986, 1: 20000}
Label counts using Counter:
Counter({0: 56986, 1: 20000})


In [7]:
lof = LocalOutlierFactor(n_neighbors=20, contamination='auto', novelty=True)

lof.fit(X_train_benign)

y_pred_lof = lof.predict(X_test_combined)  

y_pred_lof = np.array([1 if pred == -1 else 0 for pred in y_pred_lof])

print("Confusion Matrix (Local Outlier Factor):\n", confusion_matrix(y_test_combined, y_pred_lof))
print("Classification Report (Local Outlier Factor):\n", classification_report(y_test_combined, y_pred_lof))


Confusion Matrix (Local Outlier Factor):
 [[54013  2973]
 [15959  4041]]
Classification Report (Local Outlier Factor):
               precision    recall  f1-score   support

           0       0.77      0.95      0.85     56986
           1       0.58      0.20      0.30     20000

    accuracy                           0.75     76986
   macro avg       0.67      0.57      0.58     76986
weighted avg       0.72      0.75      0.71     76986

